In [3]:
import numpy as np
import open3d as o3d
import pickle

In [4]:
import sys 
from os import path
sys.path.append(path.relpath('./'))
sys.path.append(path.relpath('./CenterPoint'))

In [ ]:
print(np.__path__)
np.__version__

In [6]:
from CenterPoint.tools.visual import center_to_corner_box3d, label2color, corners_to_lines

Import spconv fail, no support for sparse convolution!
iou3d cuda not built. You don't need this if you use circle_nms. Otherwise, refer to the advanced installation part to build this cuda extension


In [7]:
with open("waymo/detection_pred.pkl", 'rb') as f:
    detection_preds_dict = pickle.load(f)

In [8]:
with open("waymo/gt_preds.pkl", 'rb') as f:
    gt_preds_dict = pickle.load(f)

In [9]:
gt_preds_dict.keys() & detection_preds_dict.keys() #no gt label for pred dicts

set()

In [10]:
len(detection_preds_dict)

29647

In [11]:
frame_name = '39847154216997509_6440_000_6460_000-1568954807924818'

In [12]:
detection_preds_dict[frame_name].keys()

dict_keys(['box3d_lidar', 'scores', 'label_preds', 'metadata'])

In [13]:
pred_detections = detection_preds_dict[frame_name]

In [14]:
pred_detections["box3d_lidar"].shape

(123, 9)

In [15]:
pred_detections["metadata"]

{'token': '39847154216997509_6440_000_6460_000-1568954807924818'}

In [16]:
pc_name = path.join("waymo/sample_test/lidar", frame_name +".pkl")
pkl_file = pickle.load(open(pc_name, 'rb'))

In [17]:
points = pkl_file["lidars"]['points_xyz']

In [18]:
pkl_file.keys()

dict_keys(['scene_name', 'frame_name', 'frame_id', 'lidars'])

In [19]:
pkl_file["frame_name"]

'39847154216997509_6440_000_6460_000_location_phx_Night_1568954807924818'

In [20]:
pred_detections.keys()

dict_keys(['box3d_lidar', 'scores', 'label_preds', 'metadata'])

In [21]:
pred_detections["box3d_lidar"].shape

(123, 9)

In [22]:
def plot_boxes(boxes, score_thresh):
    visuals =[] 
    num_det = boxes['scores'].shape[0]
    for i in range(num_det):
        score = boxes['scores'][i]
        if score < score_thresh:
            continue 

        box = boxes['box3d_lidar'][i:i+1]#.numpy()
        label = boxes['label_preds'][i]
        corner = center_to_corner_box3d(box[:, :3], box[:, 3:6], box[:, -1])[0].tolist()
        color = label2color(int(label) -1)
        visuals.append(corners_to_lines(corner, color))
    return visuals

In [33]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points[:, :3])

visual = [pcd]
visual += plot_boxes(pred_detections, 0.5)

o3d.visualization.draw_geometries(visual, )

In [24]:
pinholeCamera = o3d.io.read_pinhole_camera_intrinsic('ScreenCamera_2022-01-06-22-41-07.json')

In [31]:
def visualize_pc(pcd):
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    vis.add_geometry(pcd)
    ctrl = vis.get_view_control()
    ctrl.change_field_of_view(step=20)
    #vis.get_render_option().load_from_json("ScreenCamera_2022-01-06-20-40-29.json")
    vis.run()
    vis.destroy_window()

In [32]:
visualize_pc(pcd)

In [26]:
with open("waymo/sample_test/lidar/1376304843325714018_3420_000_3440_000-1557855910472819.pkl", "rb") as f:
    lidar_sample = pickle.load(f)

In [ ]:
with open("waymo/sampe_train/annos/10023947602400723454_1120_000_1140_000-1552440195362591.pkl", "rb") as f:
    anno_sample = pickle.load(f)

In [ ]:
lidar_sample

{'scene_name': '1376304843325714018_3420_000_3440_000',
 'frame_name': '1376304843325714018_3420_000_3440_000_location_sf_Day_1557855910472819',
 'frame_id': 0,
 'lidars': {'points_xyz': array([[-6.3597656e+01,  5.1640625e+00,  4.7059326e+00],
         [-6.3605469e+01,  5.3203125e+00,  4.7058105e+00],
         [-6.3960938e+01,  5.5058594e+00,  4.7192993e+00],
         ...,
         [-1.1648129e+01,  5.2801962e+00, -3.0060315e-01],
         [-6.0754299e+00, -8.6429052e+00, -4.2309225e-02],
         [-9.2055626e+00, -9.0502043e+00, -1.9636852e-01]], dtype=float32),
  'points_feature': array([[0.25      , 0.06440853],
         [0.25585938, 0.01171064],
         [0.12988281, 0.11710643],
         ...,
         [0.0324707 , 0.15809368],
         [0.01000977, 0.48013636],
         [0.03173828, 0.        ]], dtype=float32)}}

In [ ]:
anno_sample

{'scene_name': '10023947602400723454_1120_000_1140_000',
 'frame_name': '10023947602400723454_1120_000_1140_000_location_sf_Day_1552440195362591',
 'frame_id': 0,
 'veh_to_global': array([ 6.99058841e-01,  7.15033409e-01,  6.63036690e-03,  8.60594771e+02,
        -7.15057567e-01,  6.98982572e-01,  1.07721788e-02,  1.55686749e+04,
         3.06795685e-03, -1.22714809e-02,  9.99919996e-01, -6.51441000e+02,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  1.00000000e+00]),
 'objects': [{'id': 0,
   'name': '-U88NMYnocLWCh6iqZwj1g',
   'label': 1,
   'box': array([-2.0863535e+01,  3.0700130e+00,  9.2482859e-01,  4.2105112e+00,
           1.8452630e+00,  1.6700000e+00,  9.9386985e-04, -4.9319939e-04,
           5.7132531e-02], dtype=float32),
   'num_points': 441,
   'detection_difficulty_level': 0,
   'combined_difficulty_level': 1,
   'global_speed': array([ 0.00034206, -0.00105551], dtype=float32),
   'global_accel': array([8.7154554e-14, 2.1571289e-13], dtype=float32)},
  {'i